In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LinearRegression 
from sklearn.preprocessing import LabelEncoder
import pickle

## reading dataset

In [ ]:
data=pd.read_csv("insurance.csv")
data.head()


In [ ]:
data["region"].value_counts()

In [ ]:
data["smoker"].value_counts()


## Label Encoding for categorical feature

In [ ]:
e=LabelEncoder()
e.fit(data["sex"])
data["sex"]=e.transform(data["sex"])

In [ ]:
e.fit(data["smoker"])
data["smoker"]=e.transform(data["smoker"])

In [ ]:
e.fit(data["region"])
data["region"]=e.transform(data["region"])


In [ ]:
data.head()

In [ ]:
x=data[["age", "bmi", "children", "sex", "smoker", "region"]]
y=data['charges']

## splitting dataset 

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

## applying LinearRegression

In [ ]:
linreg = LinearRegression()
linreg.fit(x_train, y_train)


## checking accuracy

In [ ]:
predictions = linreg.predict(x_test)
linreg.score(x_test,y_test)

In [ ]:
file = open("expense_model.pkl", 'wb')
pickle.dump(linreg, file)


## buiding web app using Flask

In [ ]:
from flask import Flask, render_template, request
import pickle 
app = Flask(__name__)
model = pickle.load(open('expense_model.pkl','rb')) #read mode
@app.route("/")
def home():
    return render_template('index.html')
@app.route("/predict", methods=['GET','POST'])
def predict():
    if request.method == 'POST':
        #access the data from form
        age=int(request.form["age"])
        bmi=int(request.form["bmi"])
        children=int(request.form["children"])
        Sex=int(request.form["Sex"])
        Smoker=int(request.form["Smoker"])
        Region=int(request.form["Region"])
        input_cols=[[age, bmi, children, Sex, Smoker, Region]]
        prediction=model.predict(input_cols)
        output=round(prediction[0], 2)
        return render_template("index.html", prediction_text='Your predicted annual Healthcare Expense is $ {}'.format(output))
if __name__ == "__main__":
    app.run()